---

## 
- **Usage**: Interactive development and ad hoc analysis
- **Management**: Manged by the user via UI, CLI or API REST
- **Termination**: Manual or Auto-termination after inactivity. 120 min (default), 10 min (minimun)
- **Cost efficiency**: cost more to run when compared to job and sql warehouse pro. $0.65 per DBU classic and $0.95 per DBU serveless 
    https://www.databricks.com/product/pricing/datascience-ml

---

%md
---

##  Serveless Compute for Notebooks
- On demand
- Auto scalable
- Run SQL and Python
- No infra
- Budget Policies
---

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

print("Attempting to create cluster. Please wait...")

c = w.clusters.create_and_wait(
  cluster_name             = 'my-cluster',
  spark_version            = '13.3.x-scala2.12',
  node_type_id             = 'i3.xlarge',
  autotermination_minutes  = 15,
  num_workers              = 1
)

print(f"The cluster is now ready at " \
      f"{w.config.host}#setting/clusters/{c.cluster_id}/configuration\n")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
 
for c in w.clusters.list():
  print(c.cluster_name)

In [0]:
%sql
with node_counts as
(SELECT
    driver_node_type as node_type, 1 as node_count
  FROM
    system.compute.clusters
WHERE 
workspace_id = dataops_prd.libs.get_workspace_id()
UNION ALL
SELECT
    worker_node_type as node_type, coalesce(worker_count,max_autoscale_workers) as node_count
  FROM
    system.compute.clusters
WHERE workspace_id = dataops_prd.libs.get_workspace_id()
)
select node_type, sum(node_count) as count
FROM
    node_counts
GROUP BY ALL
ORDER BY count DESC

In [0]:
%sql
with node_counts as
(SELECT
    driver_node_type as node_type, 1 as node_count
  FROM
    system.compute.clusters
WHERE 
workspace_id = dataops_prd.libs.get_workspace_id() AND driver_instance_pool_id is null
UNION ALL
SELECT
    worker_node_type as node_type, coalesce(worker_count,max_autoscale_workers) as node_count
  FROM
    system.compute.clusters 
WHERE workspace_id = dataops_prd.libs.get_workspace_id() and worker_instance_pool_id is null
)
select node_type, sum(node_count) as count
FROM
    node_counts
GROUP BY ALL
ORDER BY count DESC

### Managing cluster logs
- **Event log**: records all significant actions related to the cluster, such as when the cluster was created, terminated, edited, scalated or encontered any erros
- **Spark UI**: interface for monitoring and debugging job execution, stages and tasks
- **Driver logs**: outputs from the notebooks and libraries